# Deep convolutional GANs (DCGANs)

__Objective:__ explore image generation with Generative Adversarial Networks (GANs) using the Bricks dataset.

__Source:__ [notebook](https://github.com/davidADSP/Generative_Deep_Learning_2nd_Edition/blob/main/notebooks/04_gan/01_dcgan/dcgan.ipynb).

In [ ]:
import sys
from datetime import datetime, timedelta
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('../modules/')

from discriminator import Discriminator

sns.set_theme()

%load_ext autoreload
%autoreload 2

## Load data

In [ ]:
def preprocess_image(image, MAX_VALUE=128.):
    """
    Standardizes the pixel vlaues of images.
    """
    return (tf.cast(image, dtype=tf.float32) - MAX_VALUE) / MAX_VALUE

In [ ]:
data_dir = '../data/dataset/'

In [ ]:
training_data = tf.keras.utils.image_dataset_from_directory(
    directory=data_dir,
    labels=None,
    color_mode="grayscale",
    shuffle=True,
    image_size=(64, 64),
    interpolation='bilinear',
    batch_size=128
)

Preprocess images.

In [ ]:
training_data = training_data.map(lambda img: preprocess_image(img))

Plot some random images.

In [ ]:
ncols = 3

fig, axs = plt.subplots(nrows=1, ncols=ncols, figsize=(14, 6))

for col in range(ncols):
    image_plot = next(iter(training_data))[0, ...]
    
    axs[col].imshow(
        image_plot,
        cmap='gray'
    )

    axs[col].grid(False)

## Build the model

### Discriminator

In [ ]:
discriminator = Discriminator()

In [ ]:
# Test on an image.
discriminator(next(iter(training_data)))

discriminator.summary()

### Generator

In [ ]:
from generator import Generator

In [ ]:
generator = Generator()

In [ ]:
# Test the generator.
test_gen_input = tf.random.uniform(shape=(1, 100))

generator(test_gen_input)

In [ ]:
generator.summary()

### Full DCGAN

In [ ]:
from gan import DCGAN

In [ ]:
dcgan_model = DCGAN(
    discriminator=discriminator,
    generator=generator,
    latent_dim=100
)

In [ ]:
g_optimizer = tf.keras.optimizers.SGD(learning_rate=1e-2)
d_optimizer = tf.keras.optimizers.SGD(learning_rate=1e-2)

dcgan_model.compile(d_optimizer=d_optimizer, g_optimizer=g_optimizer)

Test a single training step.

In [ ]:
training_step_counter = 0

time_deltas = []
training_history = []

In [ ]:
# Note: each training step is performed on one batch of training
# data, so a number (dataset_size / batch_size) of training steps
# corresponds to an epoch.
for i in range(20):
    training_step_counter += 1
    
    t_i = datetime.now()
    
    batch = next(iter(training_data))

    metrics_dict = dcgan_model.train_step(batch)

    t_f = datetime.now()

    time_deltas.append((t_f - t_i) / timedelta(seconds=1.))
    
    training_history.append(metrics_dict)

    print(
        f'Training step: {training_step_counter}'
        f' | Time delta: {time_deltas[-1]}'
        f' | Discriminator loss: {metrics_dict["d_loss"]}'
        f' | Generator loss: {metrics_dict["g_loss"]}'
    )

In [ ]:
metrics_history = tf.constant([[metrics['d_loss'].numpy(), metrics['g_loss'].numpy()] for metrics in training_history]).numpy()

fig, axs = plt.subplots(ncols=1, nrows=2, figsize=(14, 6), sharex=True)

sns.lineplot(
    x=range(metrics_history.shape[0]),
    y=metrics_history[:, 0],
    color=sns.color_palette()[0],
    label='Discriminator loss',
    ax=axs[0]
)

plt.sca(axs[0])
plt.title('Losses', fontsize=14)
plt.ylabel('Value')
plt.legend()

sns.lineplot(
    x=range(metrics_history.shape[0]),
    y=metrics_history[:, 1],
    color=sns.color_palette()[1],
    label='Generator loss',
    ax=axs[1]
)

plt.sca(axs[1])
plt.ylabel('Value')
plt.legend()
plt.xlabel('Epoch')
plt.xticks(range(metrics_history.shape[0]))

# Training time distribution.
fig = plt.figure(figsize=(14, 3))

sns.histplot(
    x=time_deltas
)

plt.title('Distribution of times for one training step', fontsize=14)
plt.xlabel('s')

**Observations:**
- At least over the first few training steps, it looks like the discriminator loss increases while the generator one decreases. This may mean that the generator is learning to create progressively better images.

## Model training

**Warning:** even one epoch may take ages on an average machine given the number of parameters!

In [ ]:
epochs = 1

training_history = dcgan_model.fit(
    x=training_data,
    epochs=epochs
)

## Image generation

Image generation works exactly as when fake images are generated during the training step: the generator works on latent vectors with shape `(latent_dim,)` and for each returns a tensor with shape `(N, N, 1)`, where `N` is the image dimension (in our case, `N=64`). The outputted pixel values are in the `[-1, 1]` range, as the original images after preprocessing.

In [ ]:
def generate_images(dcgan, n_images):
    """
    Given a DCGAN model, generates `n_images` images.
    """
    return dcgan.generator(
        tf.random.uniform(shape=(n_images, dcgan_model.latent_dim))
    )

In [ ]:
n_images = 3

generated_images = generate_images(dcgan_model, n_images)

ncols = n_images

fig, axs = plt.subplots(nrows=1, ncols=ncols, figsize=(14, 6))

for col in range(ncols):
    axs[col].imshow(
        generated_images[col, ...],
        cmap='gray'
    )

    axs[col].grid(False)